In [1]:
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout

from kerastuner import RandomSearch, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix_user(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

## Splitting Train and Test

In [9]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [10]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


## Defining function which returns the tuned model architecture

In [11]:
def c_model(hp):
    
    model = Sequential()
    
    model.add(CuDNNLSTM(units = hp.Int('units_1', 32, 256, step = 32), input_shape=(timesteps, input_dim), 
                        kernel_initializer= 'he_normal', 
                        return_sequences = True))
    
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.8, step = 0.1)))
    
    model.add(CuDNNLSTM(units = hp.Int('units_2', 32, 256, step = 32), 
                        kernel_initializer= 'he_normal'))
    
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.8, step = 0.1)))
    
    for i in range(hp.Int('num_layers', 0, 2, step = 1)):
        model.add(Dense(units=hp.Int('dense_' + str(i), 32, 512, step = 32), 
                        kernel_initializer= 'he_normal', 
                        activation='relu'))
    
    model.add(Dropout(hp.Float('dropout_3', 0.0, 0.8, step = 0.1)))
    
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer= hp.Choice('optimizer_name', ['adam','adagrad','adadelta']),
              metrics=['accuracy']) 
    
    return model


## Tuning the model's architecture and Hyper parameter using Keras Tuner

In [12]:
class MyTuner(RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
    # You can add additional HyperParameters for preprocessing and custom training loops
    # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 256, step=32)
        kwargs['epochs'] = trial.hyperparameters.Int('epochs', 5, 30, step=5)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)

# Uses same arguments as the BayesianOptimization Tuner.
tuner = MyTuner(c_model, 
                objective='val_accuracy', 
                max_trials=10, 
                executions_per_trial=10, 
                directory='Random tuner', 
                project_name='optimized_model')

# Don't pass epochs or batch_size here, let the Tuner tune them.
tuner.search(X_train, 
             Y_train, 
             validation_split=0.3, 
             verbose=2,
             use_multiprocessing=True)

tuner.search_space_summary()

tuner.results_summary()

Train on 5146 samples, validate on 2206 samples
Epoch 1/5
5146/5146 - 20s - loss: 1.0901 - accuracy: 0.5286 - val_loss: 0.6732 - val_accuracy: 0.7049
Epoch 2/5
5146/5146 - 4s - loss: 0.7284 - accuracy: 0.6901 - val_loss: 0.4658 - val_accuracy: 0.9039
Epoch 3/5
5146/5146 - 4s - loss: 0.5776 - accuracy: 0.7798 - val_loss: 0.3909 - val_accuracy: 0.9116
Epoch 4/5
5146/5146 - 4s - loss: 0.6150 - accuracy: 0.7851 - val_loss: 0.3307 - val_accuracy: 0.9266
Epoch 5/5
5146/5146 - 4s - loss: 0.5397 - accuracy: 0.8154 - val_loss: 0.4019 - val_accuracy: 0.9025
Train on 5146 samples, validate on 2206 samples
Epoch 1/5
5146/5146 - 8s - loss: 1.1154 - accuracy: 0.5307 - val_loss: 0.6526 - val_accuracy: 0.7729
Epoch 2/5
5146/5146 - 4s - loss: 0.7280 - accuracy: 0.6998 - val_loss: 0.4860 - val_accuracy: 0.8468
Epoch 3/5
5146/5146 - 4s - loss: 0.5312 - accuracy: 0.7892 - val_loss: 0.3769 - val_accuracy: 0.8998
Epoch 4/5
5146/5146 - 4s - loss: 0.4149 - accuracy: 0.8278 - val_loss: 0.4771 - val_accuracy: 0

Train on 5146 samples, validate on 2206 samples
Epoch 1/10
5146/5146 - 14s - loss: 2.0074 - accuracy: 0.1143 - val_loss: 1.9452 - val_accuracy: 0.0680
Epoch 2/10
5146/5146 - 1s - loss: 1.9867 - accuracy: 0.1238 - val_loss: 1.9291 - val_accuracy: 0.0684
Epoch 3/10
5146/5146 - 1s - loss: 1.9824 - accuracy: 0.1172 - val_loss: 1.9133 - val_accuracy: 0.0684
Epoch 4/10
5146/5146 - 1s - loss: 1.9646 - accuracy: 0.1288 - val_loss: 1.8980 - val_accuracy: 0.0707
Epoch 5/10
5146/5146 - 1s - loss: 1.9514 - accuracy: 0.1310 - val_loss: 1.8831 - val_accuracy: 0.0707
Epoch 6/10
5146/5146 - 1s - loss: 1.9328 - accuracy: 0.1403 - val_loss: 1.8680 - val_accuracy: 0.0748
Epoch 7/10
5146/5146 - 1s - loss: 1.9245 - accuracy: 0.1487 - val_loss: 1.8531 - val_accuracy: 0.0820
Epoch 8/10
5146/5146 - 1s - loss: 1.9124 - accuracy: 0.1490 - val_loss: 1.8385 - val_accuracy: 0.0975
Epoch 9/10
5146/5146 - 1s - loss: 1.8947 - accuracy: 0.1487 - val_loss: 1.8246 - val_accuracy: 0.1079
Epoch 10/10
5146/5146 - 1s - loss

Epoch 8/10
5146/5146 - 1s - loss: 1.8698 - accuracy: 0.1801 - val_loss: 1.7884 - val_accuracy: 0.1963
Epoch 9/10
5146/5146 - 1s - loss: 1.8489 - accuracy: 0.1955 - val_loss: 1.7763 - val_accuracy: 0.2480
Epoch 10/10
5146/5146 - 1s - loss: 1.8478 - accuracy: 0.1965 - val_loss: 1.7646 - val_accuracy: 0.3368
Train on 5146 samples, validate on 2206 samples
Epoch 1/10
5146/5146 - 3s - loss: 1.8120 - accuracy: 0.2198 - val_loss: 1.7449 - val_accuracy: 0.1822
Epoch 2/10
5146/5146 - 1s - loss: 1.8077 - accuracy: 0.2196 - val_loss: 1.7334 - val_accuracy: 0.1831
Epoch 3/10
5146/5146 - 1s - loss: 1.7921 - accuracy: 0.2349 - val_loss: 1.7222 - val_accuracy: 0.1840
Epoch 4/10
5146/5146 - 1s - loss: 1.7807 - accuracy: 0.2363 - val_loss: 1.7113 - val_accuracy: 0.1836
Epoch 5/10
5146/5146 - 1s - loss: 1.7632 - accuracy: 0.2380 - val_loss: 1.7006 - val_accuracy: 0.1840
Epoch 6/10
5146/5146 - 1s - loss: 1.7579 - accuracy: 0.2491 - val_loss: 1.6899 - val_accuracy: 0.1859
Epoch 7/10
5146/5146 - 1s - loss:

Train on 5146 samples, validate on 2206 samples
Epoch 1/15
5146/5146 - 22s - loss: 1.1646 - accuracy: 0.5593 - val_loss: 0.7863 - val_accuracy: 0.6963
Epoch 2/15
5146/5146 - 4s - loss: 0.7930 - accuracy: 0.6955 - val_loss: 0.6605 - val_accuracy: 0.7117
Epoch 3/15
5146/5146 - 4s - loss: 0.6787 - accuracy: 0.7347 - val_loss: 0.5842 - val_accuracy: 0.7679
Epoch 4/15
5146/5146 - 4s - loss: 0.6136 - accuracy: 0.7635 - val_loss: 0.5142 - val_accuracy: 0.8046
Epoch 5/15
5146/5146 - 4s - loss: 0.5410 - accuracy: 0.8016 - val_loss: 0.4660 - val_accuracy: 0.8599
Epoch 6/15
5146/5146 - 4s - loss: 0.4715 - accuracy: 0.8237 - val_loss: 0.4275 - val_accuracy: 0.8717
Epoch 7/15
5146/5146 - 4s - loss: 0.4000 - accuracy: 0.8663 - val_loss: 0.3795 - val_accuracy: 0.8840
Epoch 8/15
5146/5146 - 4s - loss: 0.3604 - accuracy: 0.8824 - val_loss: 0.3609 - val_accuracy: 0.8840
Epoch 9/15
5146/5146 - 4s - loss: 0.3231 - accuracy: 0.8972 - val_loss: 0.3151 - val_accuracy: 0.9093
Epoch 10/15
5146/5146 - 4s - loss

Epoch 4/15
5146/5146 - 4s - loss: 0.5583 - accuracy: 0.7960 - val_loss: 0.5928 - val_accuracy: 0.7883
Epoch 5/15
5146/5146 - 4s - loss: 0.4859 - accuracy: 0.8307 - val_loss: 0.5164 - val_accuracy: 0.8277
Epoch 6/15
5146/5146 - 4s - loss: 0.4244 - accuracy: 0.8603 - val_loss: 0.4748 - val_accuracy: 0.8518
Epoch 7/15
5146/5146 - 4s - loss: 0.3741 - accuracy: 0.8782 - val_loss: 0.4283 - val_accuracy: 0.8640
Epoch 8/15
5146/5146 - 4s - loss: 0.3419 - accuracy: 0.8935 - val_loss: 0.4623 - val_accuracy: 0.8477
Epoch 9/15
5146/5146 - 4s - loss: 0.3194 - accuracy: 0.9003 - val_loss: 0.4541 - val_accuracy: 0.8558
Epoch 10/15
5146/5146 - 4s - loss: 0.2963 - accuracy: 0.9108 - val_loss: 0.4426 - val_accuracy: 0.8590
Epoch 11/15
5146/5146 - 4s - loss: 0.2789 - accuracy: 0.9135 - val_loss: 0.5153 - val_accuracy: 0.8336
Epoch 12/15
5146/5146 - 4s - loss: 0.2646 - accuracy: 0.9213 - val_loss: 0.4709 - val_accuracy: 0.8558
Epoch 13/15
5146/5146 - 4s - loss: 0.2523 - accuracy: 0.9227 - val_loss: 0.4055

...

Epoch 12/25
5146/5146 - 1s - loss: 1.3005 - accuracy: 0.4858 - val_loss: 0.9601 - val_accuracy: 0.5766
Epoch 13/25
5146/5146 - 1s - loss: 1.2830 - accuracy: 0.4856 - val_loss: 0.9387 - val_accuracy: 0.5839
Epoch 14/25
5146/5146 - 1s - loss: 1.2437 - accuracy: 0.5105 - val_loss: 0.9047 - val_accuracy: 0.6183
Epoch 15/25
5146/5146 - 1s - loss: 1.2056 - accuracy: 0.5087 - val_loss: 0.8803 - val_accuracy: 0.6360
Epoch 16/25
5146/5146 - 1s - loss: 1.1640 - accuracy: 0.5243 - val_loss: 0.8758 - val_accuracy: 0.6228
Epoch 17/25
5146/5146 - 1s - loss: 1.1376 - accuracy: 0.5424 - val_loss: 0.8354 - val_accuracy: 0.6578
Epoch 18/25
5146/5146 - 1s - loss: 1.1199 - accuracy: 0.5468 - val_loss: 0.8221 - val_accuracy: 0.6600
Epoch 19/25
5146/5146 - 1s - loss: 1.0946 - accuracy: 0.5544 - val_loss: 0.8056 - val_accuracy: 0.6723
Epoch 20/25
5146/5146 - 1s - loss: 1.0775 - accuracy: 0.5639 - val_loss: 0.7948 - val_accuracy: 0.6763
Epoch 21/25
5146/5146 - 1s - loss: 1.0573 - accuracy: 0.5721 - val_loss: 

Train on 5146 samples, validate on 2206 samples
Epoch 1/20
5146/5146 - 15s - loss: 1.0539 - accuracy: 0.5562 - val_loss: 0.6683 - val_accuracy: 0.7548
Epoch 2/20
5146/5146 - 2s - loss: 0.5547 - accuracy: 0.7639 - val_loss: 0.5011 - val_accuracy: 0.8445
Epoch 3/20
5146/5146 - 2s - loss: 0.2810 - accuracy: 0.9135 - val_loss: 0.5739 - val_accuracy: 0.8849
Epoch 4/20
5146/5146 - 2s - loss: 0.1741 - accuracy: 0.9341 - val_loss: 0.4292 - val_accuracy: 0.9116
Epoch 5/20
5146/5146 - 2s - loss: 0.1630 - accuracy: 0.9320 - val_loss: 0.5974 - val_accuracy: 0.8971
Epoch 6/20
5146/5146 - 2s - loss: 0.1689 - accuracy: 0.9378 - val_loss: 0.4799 - val_accuracy: 0.9139
Epoch 7/20
5146/5146 - 2s - loss: 0.1282 - accuracy: 0.9415 - val_loss: 0.5266 - val_accuracy: 0.9180
Epoch 8/20
5146/5146 - 2s - loss: 0.1579 - accuracy: 0.9384 - val_loss: 0.4040 - val_accuracy: 0.9243
Epoch 9/20
5146/5146 - 2s - loss: 0.1142 - accuracy: 0.9499 - val_loss: 0.4492 - val_accuracy: 0.9297
Epoch 10/20
5146/5146 - 2s - loss

Epoch 19/20
5146/5146 - 2s - loss: 0.1088 - accuracy: 0.9448 - val_loss: 0.5864 - val_accuracy: 0.9170
Epoch 20/20
5146/5146 - 2s - loss: 0.0866 - accuracy: 0.9528 - val_loss: 0.5570 - val_accuracy: 0.9102
Train on 5146 samples, validate on 2206 samples
Epoch 1/20
5146/5146 - 7s - loss: 1.0320 - accuracy: 0.5637 - val_loss: 0.6382 - val_accuracy: 0.7947
Epoch 2/20
5146/5146 - 2s - loss: 0.4163 - accuracy: 0.8399 - val_loss: 0.4591 - val_accuracy: 0.8785
Epoch 3/20
5146/5146 - 2s - loss: 0.2133 - accuracy: 0.9229 - val_loss: 0.4180 - val_accuracy: 0.9125
Epoch 4/20
5146/5146 - 2s - loss: 0.1546 - accuracy: 0.9413 - val_loss: 0.6201 - val_accuracy: 0.8726
Epoch 5/20
5146/5146 - 2s - loss: 0.1780 - accuracy: 0.9302 - val_loss: 0.3787 - val_accuracy: 0.9207
Epoch 6/20
5146/5146 - 2s - loss: 0.1104 - accuracy: 0.9506 - val_loss: 0.5022 - val_accuracy: 0.8962
Epoch 7/20
5146/5146 - 2s - loss: 0.1092 - accuracy: 0.9518 - val_loss: 0.4852 - val_accuracy: 0.8944
Epoch 8/20
5146/5146 - 2s - loss

Epoch 17/20
5146/5146 - 2s - loss: 0.1613 - accuracy: 0.9353 - val_loss: 0.4736 - val_accuracy: 0.9161
Epoch 18/20
5146/5146 - 2s - loss: 0.1121 - accuracy: 0.9518 - val_loss: 0.5615 - val_accuracy: 0.9112
Epoch 19/20
5146/5146 - 2s - loss: 0.0954 - accuracy: 0.9551 - val_loss: 0.5326 - val_accuracy: 0.9288
Epoch 20/20
5146/5146 - 2s - loss: 0.1143 - accuracy: 0.9518 - val_loss: 0.6096 - val_accuracy: 0.9012
Train on 5146 samples, validate on 2206 samples
Epoch 1/20
5146/5146 - 7s - loss: 1.0149 - accuracy: 0.5729 - val_loss: 0.7199 - val_accuracy: 0.7185
Epoch 2/20
5146/5146 - 2s - loss: 0.5132 - accuracy: 0.7928 - val_loss: 0.6621 - val_accuracy: 0.8413
Epoch 3/20
5146/5146 - 2s - loss: 0.2779 - accuracy: 0.9034 - val_loss: 0.5178 - val_accuracy: 0.8944
Epoch 4/20
5146/5146 - 2s - loss: 0.1844 - accuracy: 0.9310 - val_loss: 0.5432 - val_accuracy: 0.8926
Epoch 5/20
5146/5146 - 2s - loss: 0.1471 - accuracy: 0.9409 - val_loss: 0.7497 - val_accuracy: 0.8753
Epoch 6/20
5146/5146 - 2s - lo

INFO:tensorflow:Oracle triggered exit


## Best hyper parameters

In [13]:
best_hp = tuner.get_best_hyperparameters()[0]
best_hp.values

{'units_1': 224,
 'dropout_1': 0.8000000000000003,
 'units_2': 96,
 'dropout_2': 0.30000000000000004,
 'num_layers': 1,
 'dropout_3': 0.6000000000000001,
 'optimizer_name': 'adam',
 'batch_size': 64,
 'epochs': 20,
 'dense_0': 288}

## Best Model's Architecture

In [14]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 128, 224)          210560    
_________________________________________________________________
dropout (Dropout)            (None, 128, 224)          0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 96)                123648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 96)                0         
_________________________________________________________________
dense (Dense)                (None, 288)               27936     
_________________________________________________________________
dropout_2 (Dropout)          (None, 288)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

## Best Model on Test data

In [38]:
best_model.fit(X_train, Y_train,
          batch_size=194,
          epochs=5,
          validation_data=(X_test, Y_test),
          verbose = 2)

Train on 7352 samples, validate on 2947 samples
Epoch 1/5
7352/7352 - 1s - loss: 0.1056 - accuracy: 0.9684 - val_loss: 0.4614 - val_accuracy: 0.9342
Epoch 2/5
7352/7352 - 1s - loss: 0.0651 - accuracy: 0.9788 - val_loss: 0.5997 - val_accuracy: 0.9328
Epoch 3/5
7352/7352 - 1s - loss: 0.0472 - accuracy: 0.9807 - val_loss: 0.6456 - val_accuracy: 0.9270
Epoch 4/5
7352/7352 - 1s - loss: 0.0386 - accuracy: 0.9838 - val_loss: 0.5644 - val_accuracy: 0.9386
Epoch 5/5
7352/7352 - 1s - loss: 0.0269 - accuracy: 0.9897 - val_loss: 0.5855 - val_accuracy: 0.9410


In [39]:
score = best_model.evaluate(X_test, Y_test, verbose = 2)

2947/1 - 1s - loss: 0.2927 - accuracy: 0.9410


In [40]:
print('\n\nThe test result obtained By fine-tuning the model is :\nLoss = {}\nAccuracy is : {}'.format(score[0],score[1]))



The test result obtained By fine-tuning the model is :
Loss = 0.5854996841915335
Accuracy is : 0.9409568905830383


In [41]:
# Confusion Matrix
confusion_matrix_user(Y_test, best_model.predict(X_test, batch_size=192))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,418,69,0,0,4
STANDING,0,45,487,0,0,0
WALKING,0,0,0,463,30,3
WALKING_DOWNSTAIRS,0,0,0,1,415,4
WALKING_UPSTAIRS,0,0,0,12,6,453


# Summary

* Performed hyper parameter tuning with different architectures using Keras Tuner
* Without hyper parameter tuning and a single layer LSTM, an Accuracy of 90 % was obtained
* By performing hyper parameter tuning and an advance architecture, best Accuracy observed is 94.09 %
* Tuning the model resulted in an improvement of model's accuracy by 4.09 %